In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import requests
from bs4 import BeautifulSoup
import pandas as pd
# importing libraries to plot the wordcloud
#from wordcloud import WordCloud, STOPWORDS
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt


In [2]:
local = "São Paulo"
email = 'caiocezar05@gmail.com'
senha = 'psx36547'


In [3]:
def login(email,senha):
    driver = webdriver.Chrome(r"C:\Users\a92550\Downloads\chromedriver.exe")
    driver.get('https://www.linkedin.com/login')
    # waiting load
    time.sleep(2)

    # Search for login and password inputs, send credentions
    driver.find_element_by_id('username').send_keys(email)
    driver.find_element_by_id('password').send_keys(senha)
    try:
        driver.find_element_by_XPATH('/html/body/div/div[1]/section/div[2]/div/article/footer/div/div/button[1]').click()
    except: pass

    driver.find_element_by_id('password').send_keys(Keys.RETURN)
    return driver


In [4]:

def job_Search_complete(driver,cargo,local,npages):
    
    driver.get(
    f"https://www.linkedin.com/jobs/search/?currentJobId=2662929045&geoId=106057199&keywords={cargo}&location={local}")
    # waiting load
    time.sleep(5)

    data = {'job':[],'description':[]}

    # each page show us some jobs, sometimes show 25, others 13 or 21 ¯\_(ツ)_/¯
    # with this knowledge I created a loop that will check how many jobs the page is listing
    # linkedin show us 40 jobs pages, then the line below will repeat 40 times
    for i in range(2, npages+1):
        try:
        # each page show us some jobs, sometimes show 25, others 13 or 21 ¯\_(ツ)_/¯
            jobs_lists = driver.find_element_by_class_name('jobs-search-results__list')  # here we create a list with jobs
            jobs = jobs_lists.find_elements_by_class_name('jobs-search-results__list-item')  # here we select each job to count
        # waiting load
            time.sleep(1)
        except:
            break
            print('isso é tudo')
        # the loop below is for the algorithm to click exactly on the number of jobs that is showing in list
        # in order to avoid errors that will stop the automation
        for n in range(1, len(jobs)+1):
            # job click
            try: 
                driver.find_element_by_xpath(f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/ul/li[{n}]/div/div/div[1]/div[2]/div[1]/a').click()
            # waiting load
                time.sleep(1)
            # select job description
                page = driver.page_source
                soup = BeautifulSoup(page,features="html.parser")
                job_desc = soup.find('div',{'class': "jobs-box__html-content jobs-description-content__text t-14 t-normal"}).find_all('span')
                
                ul = []
                head = []
                
                for span in job_desc:

                    try: data['description'].append(span.text)
                    except: data['description'].append('')
                        
                    try: data['job'].append(jobs[n].text.replace('\n','|'))
                    except: data['job'].append('')
                    
            except: pass
        # click button to change the job list
        
        try:
            driver.execute_script("arguments[0].scrollIntoView();", WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/section/div/ul/li[{i}]/button'))))
            time.sleep(5)
            driver.find_element_by_xpath( f'/html/body/div[5]/div[3]/div[3]/div[2]/div/section[1]/div/div/section/div/ul/li[{i}]/button').click()
            time.sleep(5)
        except:
            print("Last page reached")
            print(i)
            break

    # Creating a Dataframe with list
    df = pd.DataFrame(data)
    
    return df

In [5]:

def profile_Search_list(driver,cargo,npages):
    
    job = '%20'.join(cargo.split())
    
    driver.get(
        f"https://www.linkedin.com/search/results/people/?geoUrn=%5B%22106057199%22%5D&keywords={job}")
    # waiting load
    time.sleep(3)
        
    links = []
    

    for i in range(2, npages+1):
        try:
            page = driver.page_source
            soup = BeautifulSoup(page,features="html.parser")
            LinkList = soup.find('ul',{'class': "reusable-search__entity-result-list list-style-none"})

            for link in LinkList.find_all('a'):
                links.append(link.attrs['href'])
            driver.get(
            f"https://www.linkedin.com/search/results/people/?geoUrn=%5B%22106057199%22%5D&keywords={job}&page={i}")
            # waiting load
            time.sleep(3)
        
        except:
            break
            print('isso é tudo')

    return links

def get_profiles(driver,link_list):
    
    data = {'sobre':[],'experiencia':[],'formação':[]}
    
    for link in link_list:
        driver.get(link)
        # waiting load
        time.sleep(8)
        
        try:
            page = driver.page_source
            soup = BeautifulSoup(page,features="html.parser")
            profile = soup.find('div',{'class': "profile-detail"})
        except: 
            pass
            
        try:
            sobre = profile.find('div',{'class': 'inline-show-more-text inline-show-more-text--is-collapsed mt4 t-14'}).text #sobre
            data['sobre'].append(sobre.replace('\n',' ').replace('  ',' '))
        except: 
            data['sobre'].append('')
            
        try:
            experiencia = profile.find('section',{'id': 'experience-section'}).text
            data['experiencia'].append(experiencia.replace('\n',' ').replace('  ',' '))
        except: 
            data['experiencia'].append('')
            
        try:
            formação = profile.find('section',{'id': 'education-section'}).text
            data['formação'].append(formação.replace('\n',' ').replace('  ',' '))
        except: 
            data['formação'].append('')   
    return data

In [8]:
driver = login(email,senha)

In [9]:
Fiscal = job_Search_complete(driver,'Analista de fiscal',local,npages=100)

Fiscal['cargo'] = Fiscal['job'].str.split('|',expand=True)[0]
Fiscal['empresa'] = Fiscal['job'].str.split('|',expand=True)[1]
Fiscal['local'] = Fiscal['job'].str.split('|',expand=True)[2]

Fiscal.to_excel('LinkedIn Fiscal.xlsx')

Last page reached
11


In [15]:
Contabilidade = job_Search_complete(driver,'Analista de Contabilidade',local,npages=100)

Contabilidade['cargo'] = Contabilidade['job'].str.split('|',expand=True)[0]
Contabilidade['empresa'] = Contabilidade['job'].str.split('|',expand=True)[1]
Contabilidade['local'] = Contabilidade['job'].str.split('|',expand=True)[2]

Contabilidade.to_excel('LinkedIn Contabilidade.xlsx')

Last page reached
7


In [ ]:
Tributário = job_Search_complete(driver,'Analista tributário',local,npages=100)

Tributário['cargo'] = Data_analytics['job'].str.split('|',expand=True)[0]
Tributário['empresa'] = Data_analytics['job'].str.split('|',expand=True)[1]
Tributário['local'] = Data_analytics['job'].str.split('|',expand=True)[2]

Tributário.to_excel('LinkedIn Tributário.xlsx')

In [10]:
Data_scientist = job_Search_complete(driver,'Cientista de dados',local,npages=100)

Data_scientist['cargo'] = Data_scientist['job'].str.split('|',expand=True)[0]
Data_scientist['empresa'] = Data_scientist['job'].str.split('|',expand=True)[1]
Data_scientist['local'] = Data_scientist['job'].str.split('|',expand=True)[2]

Data_scientist.to_excel('LinkedIn Data_scientist.xlsx')

Last page reached
11
